# This script imports brazilian budget execution data
Source of data: https://www.siop.planejamento.gov.br/modulo/login/index.html#/
Data collected in Jan, 25th, 2024.

Função 9 - Previdência, Função 10 - Saúde, Função 12 - Educação

GND 4 e 5 - Investimentos e inversões financeiras
GND 1 - Pessoal

Rp 1 - Obrigatórias
RP 2+ 3 - Discricionárias
Rp 6+7+8+9 - Emendas

In [1]:
import pandas as pd
from useful_functions import *

In [2]:
# Read in the data
df_budget_raw = pd.read_excel('../data/dF_budget_raw_2001-2023_25.01.2024.xlsx')


In [3]:
# Rename columns
df_budget_raw.rename(columns={'Ano': 'year','Função': 'function', 'Grupo de Despesa': 'group', 'Resultado Primário': 'type', 'Empenhado': 'spent_value','Dotação Inicial': 'initial_value' }, inplace=True)

# Extract codes for each column
df_budget_raw['function_code'] = df_budget_raw['function'].str.split(' ', n = 1).str[0]
df_budget_raw['group_code'] = df_budget_raw['group'].str.split(' ', n = 1).str[0]
df_budget_raw['type_code'] = df_budget_raw['type'].str.split(' ', n = 1).str[0]

# Reorder columns
df_budget_raw = df_budget_raw.reindex(columns=['year', 'function', 'function_code', 'group', 'group_code', 'type', 'type_code', 'initial_value', 'spent_value'])

# Adjust the values format
for value in ['initial_value', 'spent_value']:
    df_budget_raw[value] = df_budget_raw[value].str.replace('.', '').str.replace(',', '.').astype(float)


# Filter


C:\Users\augus\AppData\Local\Temp\ipykernel_26136\942927005.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_budget_raw[value] = df_budget_raw[value].str.replace('.', '').str.replace(',', '.').astype(float)


In [4]:
desired_functions = ['09', '10', '12'] #Function 9 - Pension, Function 10 - Health, Function 12 - Education

desired_groups = ['1', '4', '5'] # Group 1 - Personal, Group 4 - Investiments and group 5 - Financial changes

desired_types = ['1', '2', '3', '6', '7', '8', '9'] # Group 1 - Personal, Group 4 - Investiments and group 5 - Financial changes


In [5]:
#Create some copies of the original dataframe

# Filter by function
df_function = df_budget_raw[df_budget_raw['function_code'].isin(desired_functions)].copy()[['year', 'function', 'function_code', 'initial_value', 'spent_value']]
df_function.rename(columns={'function_code': 'code'}, inplace=True)

# Filter by group
df_group = df_budget_raw[df_budget_raw['group_code'].isin(desired_groups)].copy()[['year', 'group', 'group_code', 'initial_value', 'spent_value']]
df_group.rename(columns={'group_code': 'code'}, inplace=True)

# Filter by type
df_type = df_budget_raw[df_budget_raw['type_code'].isin(desired_types)].copy()[['year', 'type', 'type_code', 'initial_value', 'spent_value']]
df_type.rename(columns={'type_code': 'code'}, inplace=True)


In [6]:
# Some adjustments for function df

# Group df_function by year and sum the values
df_step= df_function.groupby(['year', 'code']).sum().reset_index()

# Create a new dictionary to store the filtered dataframes
filtered_dfs = {}

# Loop through the types of values to create the filtered dataframes
for i, value in enumerate(['initial_value', 'spent_value']):
    
    filtered_dfs[i]  = df_step[['year', 'code', value]].copy().pivot(index='year', columns='code', values=value) # filter and pivot the dataframe and them store it in the dictionary
    
    for column in filtered_dfs[i].columns:
        filtered_dfs[i].rename(columns={column: f"bud_fun_{column}_{value}"}, inplace=True) # rename the columns to avoid confusion

df_function = pd.concat(filtered_dfs.values(), axis=1) # concatenate the dataframes from the dictionary




C:\Users\augus\AppData\Local\Temp\ipykernel_26136\3598573221.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_step= df_function.groupby(['year', 'code']).sum().reset_index()


code,bud_fun_09_initial_value,bud_fun_10_initial_value,bud_fun_12_initial_value,bud_fun_09_spent_value,bud_fun_10_spent_value,bud_fun_12_spent_value
year,,,,,,
2000,8.650877e+10,1.807792e+10,8.930709e+09,9.340757e+10,2.027012e+10,9.648482e+09
2001,1.020158e+11,2.364010e+10,1.255176e+10,6.171883e+10,2.356345e+10,4.161812e+09
2002,1.147269e+11,2.588581e+10,1.357985e+10,1.232181e+11,2.543464e+10,1.260627e+10
2003,1.318386e+11,2.777500e+10,1.377210e+10,1.454776e+11,2.717183e+10,1.353341e+10
2004,1.607077e+11,3.309537e+10,1.301501e+10,1.655094e+11,3.297064e+10,1.387247e+10


In [7]:
# Some adjustments for group df

# Group df_group by year and sum the values
df_step= df_group.groupby(['year', 'code']).sum().reset_index()

# Create a new dictionary to store the filtered dataframes
filtered_dfs = {}

# Loop through the types of values to create the filtered dataframes
for i, value in enumerate(['initial_value', 'spent_value']):
    
    filtered_dfs[i]  = df_step[['year', 'code', value]].copy().pivot(index='year', columns='code', values=value) # filter and pivot the dataframe and them store it in the dictionary
    
    for column in filtered_dfs[i].columns:
        filtered_dfs[i].rename(columns={column: f"bud_group_{column}_{value}"}, inplace=True) # rename the columns to avoid confusion

# Concatenate the dataframes from the dictionary
df_group = pd.concat(filtered_dfs.values(), axis=1)

# Create new columns to store the sum of the initial values from the groups 4 and 5
df_group['bud_group_invest_initial_value'] = df_group[['bud_group_4_initial_value', 'bud_group_5_initial_value']].sum(axis=1)
df_group.drop(['bud_group_4_initial_value', 'bud_group_5_initial_value'], axis=1, inplace=True)

# Create new columns to store the sum of the spent values from the groups 4 and 5
df_group['bud_group_invest_spent_value'] = df_group[['bud_group_4_spent_value', 'bud_group_5_spent_value']].sum(axis=1)
df_group.drop(['bud_group_4_spent_value', 'bud_group_5_spent_value'], axis=1, inplace=True)

# Rename columns
df_group.rename(columns={'bud_group_1_initial_value': 'bud_group_personal_initial_value',
                        'bud_group_1_spent_value': 'bud_group_personal_spent_value'}, inplace=True)



C:\Users\augus\AppData\Local\Temp\ipykernel_26136\3425800895.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_step= df_group.groupby(['year', 'code']).sum().reset_index()


code,bud_group_personal_initial_value,bud_group_personal_spent_value,bud_group_invest_initial_value,bud_group_invest_spent_value
year,,,,
2000,5.208682e+10,5.824445e+10,1.454512e+10,1.206392e+10
2001,5.948368e+10,6.267857e+10,2.054504e+10,1.544212e+10
2002,6.849781e+10,7.502904e+10,1.840529e+10,1.843872e+10
2003,7.704619e+10,7.899434e+10,1.482049e+10,7.108842e+09
2004,8.412004e+10,8.943218e+10,1.307009e+10,1.209923e+10


In [8]:
# Some adjustments for type df

# Group df_type by year and sum the values
df_step= df_type.groupby(['year', 'code']).sum().reset_index()

# Create a new dictionary to store the filtered dataframes
filtered_dfs = {}

#   Loop through the types of values to create the filtered dataframes
for i, value in enumerate(['initial_value', 'spent_value']):
    
    filtered_dfs[i]  = df_step[['year', 'code', value]].copy().pivot(index='year', columns='code', values=value) # filter and pivot the dataframe and them store it in the dictionary
    
    for column in filtered_dfs[i].columns:
        filtered_dfs[i].rename(columns={column: f"bud_type_{column}_{value}"}, inplace=True) # rename the columns to avoid confusion

# Concatenate the dataframes from the dictionary
df_type = pd.concat(filtered_dfs.values(), axis=1)

# Create new columns to store the sum of the initial values from the types 6, 7, 8 and 9
df_type['bud_type_amendments_initial_value'] = df_type[['bud_type_6_initial_value', 'bud_type_7_initial_value', 'bud_type_8_initial_value', 'bud_type_9_initial_value']].sum(axis=1)
df_type['bud_type_disc_initial_value'] = df_type[['bud_type_2_initial_value', 'bud_type_3_initial_value']].sum(axis=1)
df_type.drop(['bud_type_6_initial_value', 'bud_type_7_initial_value', 'bud_type_8_initial_value', 'bud_type_9_initial_value', 'bud_type_2_initial_value', 'bud_type_3_initial_value'], axis=1, inplace=True)

# Create new columns to store the sum of the spent values from the types 6, 7, 8 and 9
df_type['bud_type_amendments_spent_value'] = df_type[['bud_type_6_spent_value', 'bud_type_7_spent_value', 'bud_type_8_spent_value', 'bud_type_9_spent_value']].sum(axis=1)
df_type['bud_type_disc_spent_value'] = df_type[['bud_type_2_spent_value', 'bud_type_3_spent_value']].sum(axis=1)
df_type.drop(['bud_type_6_spent_value', 'bud_type_7_spent_value', 'bud_type_8_spent_value', 'bud_type_9_spent_value', 'bud_type_2_spent_value', 'bud_type_3_spent_value'], axis=1, inplace=True)

# Rename columns
df_type.rename(columns={'bud_type_1_initial_value': 'bud_type_mandatory_initial_value',
                        'bud_type_1_spent_value': 'bud_type_mandatory_spent_value'}, inplace=True)



C:\Users\augus\AppData\Local\Temp\ipykernel_26136\254997744.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_step= df_type.groupby(['year', 'code']).sum().reset_index()


code,bud_type_mandatory_initial_value,bud_type_mandatory_spent_value,bud_type_amendments_initial_value,bud_type_disc_initial_value,bud_type_amendments_spent_value,bud_type_disc_spent_value
year,,,,,,
2000,1.756119e+11,1.904602e+11,0.0,3.913662e+10,0.0,3.140586e+10
2001,2.084962e+11,1.363717e+11,0.0,5.009274e+10,0.0,3.547187e+10
2002,2.436230e+11,2.532746e+11,0.0,4.635984e+10,0.0,4.652082e+10
2003,2.732669e+11,2.911544e+11,0.0,5.119235e+10,0.0,3.334158e+10
2004,3.282344e+11,3.361497e+11,0.0,4.196407e+10,0.0,4.054787e+10


In [13]:
# Concatenate the dataframes
df_budget = pd.concat([df_function, df_group, df_type], axis=1)

# Convert time index to datetime and adjust its format.
df_budget.index = pd.to_datetime(df_budget.index, format='%Y')


# Change the day of the index to the last day of each year, since the data corresponds to the end of each year
#df_budget.index = pd.to_datetime(df_budget.index) + pd.DateOffset(years=1) - pd.DateOffset(days=1)
df_budget.index = pd.to_datetime(df_budget.index) + pd.DateOffset(month=12)

# # Convert the index to the desired format
df_budget.index = df_budget.index.strftime('%d-%m-%Y')

# Adjust the index name
df_budget.rename_axis('Time', inplace=True)

# Export the dataframe to an excel file
df_budget.to_csv('../data/df_budget.csv')


C:\Users\augus\AppData\Local\Temp\ipykernel_26136\1978663657.py:10: PerformanceWarning: Non-vectorized DateOffset being applied to Series or DatetimeIndex.
  df_budget.index = pd.to_datetime(df_budget.index) + pd.DateOffset(month=12)
